### download the rolling hindcasts for all GCMs from the Copernicus Climate Datastore

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from subprocess import call 
from shutil import which 
import pathlib
from datetime import datetime
from dateutil.relativedelta import relativedelta
import dateparser
import yaml
from yaml.loader import SafeLoader

In [3]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

In [4]:
import numpy as np

In [5]:
HOME = pathlib.Path.home()

In [6]:
CWD = pathlib.Path.cwd()

In [7]:
CWD

PosixPath('/home/nicolasf/operational/ICU/development/hotspots/code/ICU_Water_Watch/notebooks/C3S')

### import local functions for the downloading of the C3S forecasts 

In [8]:
sys.path.append('../..')

In [9]:
from ICU_Water_Watch import C3S, domains

### parameters for papermill

In [10]:
provider = 'CDS' # should not change
varname = 'tprate' # should not change
domain_name = 'C3S_download'
# if one wants to download other reforecasts than the ones corresponding to the current month
lag = 1
# path where to save the hindcasts / reforecasts
gcm_path = f'/media/nicolasf/END19101/ICU/data/{provider}/operational/hindcasts'
config_yaml = './CDS_config.yaml' 

### reads the yaml file, mapping GCM to system 

In [11]:
# Open the file and load the file
with open(config_yaml) as f:
    dict_systems = yaml.load(f, Loader=SafeLoader)

In [12]:
dict_systems

{'ECMWF': 5,
 'UKMO': 601,
 'METEO_FRANCE': 8,
 'DWD': 21,
 'CMCC': 35,
 'NCEP': 2,
 'JMA': 3,
 'ECCC_GEM_NEMO': 2,
 'ECCC_CanCM4i': 3}

### casts the paths to `pathlib.Path` objects 

In [13]:
gcm_path = pathlib.Path(gcm_path)

### get today's date 

In [14]:
date = dateparser.parse('today')

In [15]:
date

datetime.datetime(2022, 5, 2, 10, 28, 34, 927679)

### apply lag 

In [16]:
date = date - relativedelta(months=lag)

In [17]:
print(f"the hindcast data will be downloaded for all re-forecasts initialised in {date:%B}")

the hindcast data will be downloaded for all re-forecasts initialised in April


### generate the dates for the download of the rolling hindcasts 

In [18]:
dates = [datetime(y, date.month, 1) for y in range(1993, 2016 + 1)]

In [19]:
dates

[datetime.datetime(1993, 4, 1, 0, 0),
 datetime.datetime(1994, 4, 1, 0, 0),
 datetime.datetime(1995, 4, 1, 0, 0),
 datetime.datetime(1996, 4, 1, 0, 0),
 datetime.datetime(1997, 4, 1, 0, 0),
 datetime.datetime(1998, 4, 1, 0, 0),
 datetime.datetime(1999, 4, 1, 0, 0),
 datetime.datetime(2000, 4, 1, 0, 0),
 datetime.datetime(2001, 4, 1, 0, 0),
 datetime.datetime(2002, 4, 1, 0, 0),
 datetime.datetime(2003, 4, 1, 0, 0),
 datetime.datetime(2004, 4, 1, 0, 0),
 datetime.datetime(2005, 4, 1, 0, 0),
 datetime.datetime(2006, 4, 1, 0, 0),
 datetime.datetime(2007, 4, 1, 0, 0),
 datetime.datetime(2008, 4, 1, 0, 0),
 datetime.datetime(2009, 4, 1, 0, 0),
 datetime.datetime(2010, 4, 1, 0, 0),
 datetime.datetime(2011, 4, 1, 0, 0),
 datetime.datetime(2012, 4, 1, 0, 0),
 datetime.datetime(2013, 4, 1, 0, 0),
 datetime.datetime(2014, 4, 1, 0, 0),
 datetime.datetime(2015, 4, 1, 0, 0),
 datetime.datetime(2016, 4, 1, 0, 0)]

### path to download the rolling hindcasts 

In [20]:
if not(gcm_path.exists()): 
    gcm_path.mkdir(parents=True)

### Now loops over the dates, creates a dictionnary containing the keywords arguments, and pass to dask for parallel download 

In [ ]:
for date in dates: 
    
    downloaded_files = []

    for GCM in dict_systems.keys():
        
        if 'ECCC' in GCM: 
            
            GCM_API_name = GCM.split('_')[0]
            
            GCM_path = gcm_path.joinpath(GCM).joinpath(varname.upper())
            
            print(f"processing ECCC, path should be {str(GCM_path)}")
        
            args = dict(GCM=GCM_API_name,
            system=dict_systems[GCM],
            varname=varname,
            year=date.year,
            month=date.month,
            leadtimes=[1, 2, 3, 4, 5, 6],
            opath=GCM_path,
            domain=domains.domains[domain_name],
            file_format='netcdf',
            level='surface',
            max_retry=3)
        
        else: 

            args = dict(GCM=GCM,
            system=dict_systems[GCM],
            varname=varname,
            year=date.year,
            month=date.month,
            leadtimes=[1, 2, 3, 4, 5, 6],
            opath=gcm_path.joinpath(GCM).joinpath(varname.upper()),
            domain=domains.domains[domain_name],
            file_format='netcdf',
            level='surface',
            max_retry=3)
            

        downloaded_files.append(delayed(C3S.download)(**args))

    with ProgressBar():
        downloaded_files = compute(downloaded_files)

processing ECCC, path should be /media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/ECCC_GEM_NEMO/TPRATE
processing ECCC, path should be /media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/ECCC_CanCM4i/TPRATE
[                                        ] | 0% Completed |  0.0s
/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/CMCC/TPRATE/ensemble_seas_forecasts_tprate_from_1993_04_CMCC.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/NCEP/TPRATE/ensemble_seas_forecasts_tprate_from_1993_04_NCEP.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/METEO_FRANCE/TPRATE/ensemble_seas_forecasts_tprate_from_1993_04_METEO_FRANCE.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/ECMWF/TPRATE/ensemble_seas_forecasts_tprate_from_1993_04_ECM

2022-05-02 10:28:37,742 INFO Welcome to the CDS
2022-05-02 10:28:37,743 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-05-02 10:28:37,743 INFO Welcome to the CDS
2022-05-02 10:28:37,745 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-05-02 10:28:37,748 INFO Welcome to the CDS
2022-05-02 10:28:37,749 INFO Welcome to the CDS
2022-05-02 10:28:37,749 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-05-02 10:28:37,749 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-05-02 10:28:37,750 INFO Welcome to the CDS
2022-05-02 10:28:37,754 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-05-02 10:28:37,756 INFO Welcome to the CDS
2022-05-02 10:28:37,756 INFO Welcome to the CDS
2022-05-02 10:

[                                        ] | 0% Completed |  2.8s

2022-05-02 10:28:38,142 INFO Request is queued
2022-05-02 10:28:38,144 INFO Request is queued
2022-05-02 10:28:38,146 INFO Request is queued
2022-05-02 10:28:38,147 INFO Request is queued
2022-05-02 10:28:38,148 INFO Request is queued
2022-05-02 10:28:38,149 INFO Request is queued
2022-05-02 10:28:38,151 INFO Request is queued
2022-05-02 10:28:38,152 INFO Request is queued
2022-05-02 10:28:38,153 INFO Request is queued


[                                        ] | 0% Completed |  1min 21.6s

...

[####                                    ] | 11% Completed |  1min 40.0s


 25%|██████████████████████████▏                                                                             | 788k/3.05M [00:17<01:05, 36.6kB/s]

 17%|█████████████████▋                                                                                      | 190k/1.09M [00:17<01:19, 12.0kB/s]




  4%|████▏                                                                                                   | 180k/4.36M [00:17<03:20, 21.9kB/s]

[####                                    ] | 11% Completed |  1min 40.1s







  3%|███                                                                                                    | 94.0k/3.05M [00:16<06:48, 7.59kB/s]

[####                                    ] | 11% Completed |  1min 40.2s

### convert the notebook to HTML 

### Note: this does not work with papermill

In [ ]:
# jupyter = which('jupyter')

# nb_name = ipynbname.name()

# nb_name

# cmd = f"{jupyter} nbconvert --to html {nb_name}.ipynb"

# r = call(cmd, shell=True)